In [2]:
import json
import os
import sys
import urllib.request
import yaml
# import utils
import argparse

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
# !pip install webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
import selenium
selenium.__version__

'3.141.0'

In [4]:
selectors = {
  "facebook_https_prefix": "https://",
  "facebook_link_body": ".facebook.com/",
  }

In [5]:
facebook_https_prefix = selectors.get("facebook_https_prefix")
facebook_link_body = selectors.get("facebook_link_body")

In [6]:
driver = None

In [7]:
def init():
        global driver

        options = Options()

        #  Code to disable notifications pop up of Chrome Browser
        options.add_argument("--disable-notifications")
        options.add_argument("--disable-infobars")
        options.add_argument("--mute-audio")
        # options.add_argument("headless")

        try:
            driver = webdriver.Chrome(
                executable_path=ChromeDriverManager().install(), options=options
            )
            
        except Exception:
            print("Error loading chrome webdriver " + sys.exc_info()[0])
            exit(1)

In [8]:
def login(email, password):
    """ Logging into our own profile """

    try:
        

        fb_path = facebook_https_prefix + facebook_link_body
#         print('ok')
        driver.get(fb_path)
        driver.maximize_window()

        # filling the form
        driver.find_element_by_name("email").send_keys(email)
        driver.find_element_by_name("pass").send_keys(password)

        try:
            # clicking on login button
            driver.find_element_by_id("loginbutton").click()
        except NoSuchElementException:
            # Facebook new design
            driver.find_element_by_name("login").click()


    except Exception:
        print("There's some error in log in.")
        print(sys.exc_info()[0])
        exit(1)


In [9]:
def scroll(total_scrolls, driver, scroll_time):
    global old_height
    current_scrolls = 0

    while True:
        try:
            if current_scrolls == total_scrolls:
                return

            old_height = driver.execute_script("return document.body.scrollHeight")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            WebDriverWait(driver, scroll_time, 0.05).until(
                lambda driver: check_height(driver, old_height)
            )
            current_scrolls += 1
        except TimeoutException:
            break

    return

In [10]:
def loadlink(link):
    driver.get(link)

In [11]:
# check if height changed
def check_height(driver, old_height):
    new_height = driver.execute_script("return document.body.scrollHeight")
    return new_height != old_height

In [12]:
init()
email = ''
password = ''
login(email,password)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240


[WDM] - Driver [C:\Users\roey3\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [13]:
fbUser = ''
loadlink('https://www.facebook.com/'+fbUser)

In [14]:
scroll(3,driver,10)

In [16]:
html = driver.execute_script("return document.documentElement.outerHTML;")

In [20]:
with open(f"fb{fbUser}.html", 'w', encoding='utf-8') as f:
    f.write(html)
f.close()